In [1]:
import cv2 as cv
import numpy as np

In [2]:
def showimage(image,label = "image"):
    cv.imshow(label,image)
    cv.waitKey()
    cv.destroyAllWindows()

In [3]:
def histogram_eq(image:np.ndarray,bin = 256)->np.ndarray:
    
    
    intensity, a,b = cv.split(cv.cvtColor(image,cv.COLOR_BGR2LAB))
    ## Find the histogram of the intensities of each pixels within a bin size of 256
    hist, _ = np.histogram(intensity, bins=bin)
    cumulative_sum = hist.cumsum()
    
    ## Gets the lowest nonzero element
    sort = cumulative_sum.argsort()
    cummin = cumulative_sum[sort[0]] if cumulative_sum[sort[0]]!=0 else cumulative_sum[sort[1]]
    # cummin = cumulative_sum[cumulative_sum.nonzero()][cumulative_sum[cumulative_sum.nonzero()].argsort()[0]]
    
    ## Perform the equation found on wikipedia
    equ_intensities = np.round(((cumulative_sum[intensity.flatten()]-cummin)/((image.shape[0]*image.shape[1])-cummin)*255))
    ## Reshape back into the shape of the image
    new_intensities = equ_intensities.reshape((image.shape[0],image.shape[1])).astype(np.uint8)
    ## Merge the intensities back with the A,B values
    new_image = cv.merge([new_intensities,a,b])
    ## Convert back to BGR Space
    new_image = cv.cvtColor(new_image,cv.COLOR_LAB2BGR)
    
    return new_image

In [4]:
def bilinear_interpolate(im, x, y):
    x = np.asarray(x)
    y = np.asarray(y)

    x0 = np.floor(x).astype(int)
    x1 = x0 + 1
    y0 = np.floor(y).astype(int)
    y1 = y0 + 1

    x0 = np.clip(x0, 0, im.shape[1]-1);
    x1 = np.clip(x1, 0, im.shape[1]-1);
    y0 = np.clip(y0, 0, im.shape[0]-1);
    y1 = np.clip(y1, 0, im.shape[0]-1);

    Ia = im[ y0, x0 ]
    Ib = im[ y1, x0 ]
    Ic = im[ y0, x1 ]
    Id = im[ y1, x1 ]

    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    return (Ia.T*wa).T + (Ib.T*wb).T + (Ic.T*wc).T + (Id.T*wd).T

In [5]:
def adaptive_eq(image:np.ndarray, tile_size = 8):
    row_tiles = np.array_split(image,image.shape[0]/tile_size,axis=0)
    eq_image = []
    for row_tile in row_tiles:
        tiles = np.array_split(row_tile, row_tile.shape[1]/tile_size,axis=1)
        row_image = histogram_eq(tiles[0])
        for tile in tiles[1:]:
            eq_tile = histogram_eq(tile)
            row_image = np.hstack((row_image,eq_tile))
        eq_image.append(row_image)
    new_image = np.vstack(eq_image)
    boundries_x, boundries_y = list(range(0,new_image.shape[0]-1,tile_size)),list(range(0,new_image.shape[1]-1,tile_size))
    for x in boundries_x:
            new_image[x,boundries_y] = bilinear_interpolate(new_image,x,boundries_y)
    # intensities = bilinear_interpolate(new_image,boundries_x,boundries_y)
    # new_image[boundries_x,boundries_y] = intensities
    return new_image

def adaptive_eq(image:np.ndarray, tile_size = 8):
    # row_tiles = np.array_split(image,image.shape[0]/tile_size,axis=0)
    sliding_view = np.lib.stride_tricks.sliding_window_view(image,(tile_size,tile_size,3))
    eq_image = []
    for row_tile in sliding_view:
        # tiles = np.array_split(row_tile, row_tile.shape[1]/tile_size,axis=1)
        row_image = histogram_eq(row_tile[0,0])[:,0]
        for tile in row_tile[1:]:
            eq_tile = histogram_eq(tile[0])
            row_image = np.hstack((row_image,eq_tile[:,0]))
        row_image = np.hstack((row_image,histogram_eq(row_tile[-1][0])[:,-tile_size//2:]))
        eq_image.append(row_image)
    new_image = np.vstack(eq_image)
    return new_image

In [6]:
image = cv.imread('./Data/0000000000.png')
showimage(image)

In [7]:
showimage(histogram_eq(image=image))

In [8]:
intensity, a,b = cv.split(cv.cvtColor(image,cv.COLOR_BGR2LAB))

showimage(cv.cvtColor(cv.merge([cv.equalizeHist(intensity),a,b]),cv.COLOR_LAB2BGR))

In [9]:
showimage(adaptive_eq(image,tile_size=25))

In [10]:
sliding_view = np.lib.stride_tricks.sliding_window_view(image,(8,8,3))

AttributeError: module 'numpy.lib.stride_tricks' has no attribute 'sliding_window_view'

In [ ]:
sliding_view[0,0].shape

(1, 8, 8, 3)

tile_size = 8
sliding_view = np.lib.stride_tricks.sliding_window_view(image,(tile_size,tile_size,3))
eq_image = []
for row_tile in sliding_view:
    # tiles = np.array_split(row_tile, row_tile.shape[1]/tile_size,axis=1)
    row_image = histogram_eq(row_tile[0,0])[:,0].reshape((tile_size,1,3))
    for tile in row_tile[1:]:
        eq_tile = histogram_eq(tile[0])
        row_image = np.hstack((row_image,eq_tile[:,0,3].reshape((tile_size,1,3))))
    row_image = np.hstack((row_image,histogram_eq(row_tile[-1][0])[:,-tile_size//2:,3].reshape((tile_size,1,3))))
    eq_image.append(row_image)
new_image = np.vstack(eq_image)